### Step 1 : Import required libraries

In [1]:
import os
import traceback
from pathlib import Path

import pandas as pd
from osgeo import gdal
from helpers import *
from imputation import Imputation as imp
import numpy as np

### Step 2 : The code below takes the input file(raster image) and runs imputation algorithms on the raster image and stores them in the specified output folder

#### Explanation of the flow

In [12]:

#### Initialization 
# - Get input folder 
# - read each image in the folder using gdal as numpy array (Important : check the metadata of file and specify scale factor)
# - crop the image to have equal width and height so that the algorithms converge. 

#### Creation of missing pixels 
# - for each image create missing pixels (percentage of missing pixels - specified by the user if needed)
# - creates a new folder if not exisiting and saves it in a separate folder in the specified output folder. 

#### Predict missing pixels 
# - Set the parameters such as hil and iterations 
# - 
#   for each algorithm\
#         for each image\
#           run imputation\ 
#           create tiff file using CreateGeotiff function\ 
#           create a folder with algorithm name if not existing and save it in the folder
#### Create Geotiff 
# - creates geotiff from the input raster array. 

In [6]:
import os
import traceback
from pathlib import Path
from osgeo import gdal
from imputation import Imputation as imp  

class Imputation:
    def __init__(self, inputFile, scaleFactor=0.00002, outputFile=None):
        """
        Initialize the Imputation class with input parameters
        
        :param inputFile (str): Path to the input image file.
        :param scaleFactor (float, optional): Scaling factor for the image values. Default is 0.00002.
        :param outputFile (str, optional): Path to the output directory. Default is None.
     
        """
        
        self.inputFilePath = inputFile
        self.outputFilePath = outputFile
        self.scaleFactor = scaleFactor
        self.imputedArrays = []
        self.fileName = Path(self.inputFilePath).stem
        self.percent = None
        self.algos = ['siLRTC', 'haLRTC', 'CMTF', 'CMSI', 'cpALS', 'Base']
        
        try:
            # Open input file using gdal and extract relevant information
            self.data = gdal.Open(self.inputFilePath)
            self.geoTrans = self.data.GetGeoTransform()
            self.projection = self.data.GetProjection()
            self.rasterArray = self.data.ReadAsArray().T * scaleFactor
            self.imgWidth = self.rasterArray.shape[0]
            self.imgHeight = self.rasterArray.shape[1]
            
            if self.imgHeight != self.imgWidth:
                croppedShape = min(self.imgWidth, self.imgHeight)
                self.rasterArray = self.rasterArray[:croppedShape, :croppedShape, :]
            
            # Initialize the 'imp' class with rasterArray
            self.impute = imp(self.rasterArray)

        except:
            print("Unable to read input file using gdal")
            traceback.print_exc()

    def createMissingPixels(self, percent=10, method='random'):
        
        """
        Create missing pixels in the image.

        :param percent (int, optional): Percentage of missing pixels to create. Default is 10.
        :param method (str, optional): Method for creating missing pixels. Default is 'random'.
        """
        self.percent = percent
        self.mask, self.corruptImage = self.impute.synthetic_mask((100 - self.percent) / 100)
        path = str(self.outputFilePath + '/' + str(self.fileName))
        checkFolder = os.path.isdir(path)
        
        if not checkFolder:
            os.makedirs(path)
            print('created Folder', path)
        
        # Create GeoTIFF file for corruptImage
        self.CreateGeoTiff(outRaster=str(path + '/' + str(self.percent) + 'missingNew.tif'),
                           data=self.corruptImage,
                           geo_transform=self.geoTrans, projection=self.projection)

    def predictMissingPixels(self, outputFolder, algo=None, tensorRank=12):
        
        """
        Predict missing pixels using specified algorithm or all algorithms.

 
        :param outputFolder (str): Path to the output directory.
        :param algo (str, optional): Algorithm to use for prediction. Default is None.
        :param tensorRank (int, optional): Rank for tensor-based algorithms. Default is 12.
        """
        if algo is None:
            print("Running on all algorithms")
            self.results = {}
            
            if self.impute.get_corrupt_img() is None:
                self.impute.generate_mask(approximate_mask=True)
                print("Generating Approximate Mask")
            
            # Perform imputation and record runtime
            self.imputedArrays, self.algoRuntime = self.impute.impute(speed="slow", hil=0, skip_vis=0)
            
            self.evalResults = []
            self.runTime = []
            
            for i in range(len(self.algos) - 1):
                path = str(outputFolder + self.algos[i] + '/' + str(filename))
                checkFolder = os.path.isdir(path)
                
                if not checkFolder:
                    os.makedirs(path)
                    print('created Folder', path)
                
                # Create GeoTIFF files for imputed images and calculate RSE
                self.CreateGeoTiff(outRaster=str(path + '/' + str(self.percent) + self.algos[i] + 'New.tif'),
                                   data=self.imputedArrays[i],
                                   geo_transform=self.geoTrans, projection=self.projection)
                self.evalResults.append(RSE(self.rasterArray, self.imputedArrays[i]))
                self.runTime.append(self.algoRuntime[i + 1] - self.algoRuntime[i])
            
            self.runTime.append(None)
            self.evalResults.append(RSE(self.rasterArray, self.corruptImage))
            
            # Store evaluation results and runtime information
            self.results = {'fileName': [self.fileName] * len(self.algos),
                             'missingPercent': [str(self.percent)] * len(self.algos),
                             'algo': self.algos, 'RSE': self.evalResults, 'runTime': self.runTime}
            # print(self.tempDict)
            return self.results
        
        def CreateGeoTiff(self, outRaster, data, geo_transform, projection):
            """
            Create a GeoTIFF file from the provided data.

            Parameters:
                outRaster (str): Output path for the GeoTIFF file.
                data (numpy.ndarray): Image data to be written as a GeoTIFF.
                geo_transform (tuple): Geotransform information.
                projection (str): Projection information.
            """
            data = data.T
            driver = gdal.GetDriverByName('ENVI')

            no_bands, rows, cols = data.shape
            DataSet = driver.Create(outRaster, cols, rows, no_bands, gdal.GDT_Float32)
            DataSet.SetGeoTransform(geo_transform)
            DataSet.SetProjection(projection)

            for i, image in enumerate(data, 1):
                DataSet.GetRasterBand(i).WriteArray(image)

            DataSet.FlushCache()
            DataSet = None

### Step 3 :  The below code is to conduct experiments on each dataset 

- for each file in the input folder 
      for each missing percent(from 10% to 50%)
         create missing pixels 
         call imputation class function to predict missing pixels 
         get the results and store the statistics in a dataframe for further analysis 
         

In [ ]:
import pandas as pd

# Create an empty DataFrame to store results
finalDataframe = pd.DataFrame(columns=['fileName', 'missingPercent', 'algo', 'RSE', 'runTime'])

# Loop through files in the specified path
for filename in os.listdir(path):
    if filename.endswith('.IMG'):
        print(filename)
        
        # Iterate over missing percentage values (10 to 50, step 10)
        for missingPercent in range(10, 60, 10):
            filePath = os.path.join(path, filename)
            
            # Create an instance of the Imputation class
            impute = Imputation(inputFile=filePath, outputFile=path)
            
            # Create missing pixels in the image
            impute.createMissingPixels(percent=missingPercent)
            
            # Predict missing pixels using the specified algorithms
            results = impute.predictMissingPixels(outputFolder=path)
            
            # Concatenate the results DataFrame with the new results
            finalDataframe = pd.concat([finalDataframe, pd.DataFrame(results)], ignore_index=True)
        
        # print('------------------1file---------------------')

# Save the final results DataFrame to a TSV file
finalDataframe.to_csv('imputation_ChandDataset2_Results.tsv', sep='\t')


### Step 4 : Creating heat map for the above obtained results 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from osgeo import gdal



# function to calculate RMSE values of each pixel in the image
def calculate_rmse(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    rmse = np.sqrt(mse)
    return rmse


# Filepath of images 
orig_image = '/Users/bunny/PycharmProjects/Imputation/data_/MI/dataset2/Area2_MI_MAP_03_N22E196N21E197SC.tif'
predicted_image = '/Users/bunny/PycharmProjects/Imputation/data_/MI/dataset2/CMSI/Area2_MI_MAP_03_N22E196N21E197SC.tif/10CMSI.tif'


# read images using gdal and scale factor of the file 
#(The data is read as (9,1048,1048) i.e, color bands,width, height) using the gdal library in order to visualize the image using matplotlib I have transformed the image 
predicted_image = gdal.Open(predicted_image).ReadAsArray().T
original_image = gdal.Open(orig_image).ReadAsArray().T * 0.00002
original_image = original_image[:403,:403,:]


# Read it as numpy array 
original_array = np.array(original_image)
predicted_array = np.array(predicted_image)

# call the rmse function with the inputs 
rmse_array = np.sqrt(np.mean((original_array - predicted_array) ** 2, axis=-1))

# Create heat map
plt.figure(figsize=(12, 8))

plt.subplot(1, 3, 1)
plt.imshow(original_array[:, :, :3])
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(predicted_array[:, :, :3])
plt.title('Predicted Image')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(rmse_array, cmap='hot', interpolation='nearest')
plt.colorbar(label='RMSE Value')
plt.title('RMSE Heatmap')

plt.tight_layout()
plt.show()
